In [ ]:
#连接ERP的DB2数据库，获取TACAIA2（前台：ACAI25）,TACACM4(ACAC80非产成品),TACACMM(ACACMM),TACACJ5(废钢入库履历)
import ibm_db
import pandas as pd
connStr='DATABASE=BHBXA1;HOSTNAME=192.168.2.28;PORT=50000;PROTOCOL=TCPIP;UID=bhbxa1;PWD=bhbxa1'
con=ibm_db.connect(connStr,'','')


In [ ]:
import ibm_db_dbi

In [ ]:
conn=ibm_db_dbi.Connection(con)

In [163]:
tacaia2=pd.read_sql('select * from tacaia2',conn,parse_dates={'APP_TRNC_TIME':'%Y%m%d%H%M%S'})

In [169]:
from pandas.api.types import CategoricalDtype
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype(CategoricalDtype(['33','03','31','01','50','51']))
tacaia2['ACCOUNT_TITLE_ITEM'].cat.categories

Index(['33', '03', '31', '01', '50', '51'], dtype='object')

In [170]:
tacaia2.set_index(['MAT_NO','ACCOUNT_TITLE_ITEM','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True)

In [171]:
tacaia2.index.names

FrozenList(['MAT_NO', 'ACCOUNT_TITLE_ITEM', 'UNIT_CODE', 'SALE_ORDER_SUB_NO'])

In [172]:
tacaia2.sort_index(level=['MAT_NO','ACCOUNT_TITLE_ITEM','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True)

In [159]:
tacaia2.sort_index(level='ACCOUNT_TITLE_ITEM',inplace=True)

In [193]:
tacaia2=tacaia2.dropna() #剔除第一行的空行

In [194]:
tacaia2

MD2004291301                       1   
B1607078FEAEBA    33                 LA01      MD2005094001                           
                  03                 LA01      MD2005094001                           
                  31                 LA01      MD2005094001                           
B1607078FEAEBAAA  01                 LK01      MD2005094001                           
                  50                 LK01      MD2005094001                           
B1607078FEAEBAAB  01                 LK01      MD2005094001                           
B1607078FEAEBBA   33                 LA01      MD2005094001                           
                  03                 LA01      MD2005094001                           
                  31                 LA01      MD2005094001                           
B1607078FEAEBBAAA 01                 LK01      MD2005094001                           
                  50                 LK01      MD2005094001                           
B1607078FEAEBBAAB 01                 LK01      MD2005094001                           
B1607078FEAEBBB   33                 LA01      MD2005094001                           
                  03                 LA01      MD2005094001                           
                  31                 LA01      MD2005094001                           
B1607078FEAEBBBA  01                 LC02      MD2005094001                           
                  50                 LC02      FD2005290103                           
B1607078FEAEBBBB  01                 LC02      MD2005094001                           
                  50                 LC02      FD2005290103                           
B1607078FEAEBBBC  01                 LC02      MD2005094001                           
B1607078FEAEDDA   33                 LA01      MD2005094001                           
                  03                 LA01      MD2005094001                           
                  31                 LA01      MD2005094001                           
B1607078FEAEDDAAA 01                 LK01      MD2005094001                           
                  50                 LK01      MD2005094001                           
...                                                                             ...   
P2005004BAA       31                           LD2005200102                           
P2005004BAAAA     01                 LK01      LD2005200102                           
P2005004BAAAB     33                 LK01      LD2005200103                           
                  03                 LK01      LD2005200103                           
                  01                 LK01      LD2005200102                           
P2005004BAB       31                           LD2005200102                           
P2005004BABAA     01                 LK01      LD2005200102                           
P2005004BABAB     33                 LK01      LD2005200103                           
                  03                 LK01      LD2005200103                           
                  01                 LK01      LD2005200102                           
P2005004BABBA     01                 LK01      LD2005200102                           
P2005004BABBB     33                 LK01      LD2005200103                           
                  03                 LK01      LD2005200103                           
                  01                 LK01      LD2005200102                           
P2005004BABCA     01                 LK01      LD2005200102                           
P2005004BABCB     33                 LK01      LD2005200103                           
                  03                 LK01      LD2005200103                           
                  01                 LK01      LD2005200102                           
P2005004BABD      01                 LI01      LD2005200102                           
P2005004BB        33                         

In [195]:
tacaia2.index.get_level_values('MAT_NO')[0]

'B1603024QFEEGA'

In [177]:
#from numpy import nan as NA

In [196]:
#tacaia2.index.get_level_values('MAT_NO')[0]==' '  #因为排序后第一行[0]为空行，所以迭代时从第二行[1]开始（上面已删除空行，不用从第二行开始了）
for mat in tacaia2.index.get_level_values('MAT_NO'):
    print(tacaia2.loc[(mat,'31'),'KEY_SEQ'])

KeyError: ('B1603024QFEEGA', '31')

In [181]:
tacaia2.to_excel('tacaia2-202005.xlsx')

In [ ]:
#tacaia2.to_excel('tacaia2-202005.xlsx',sheet_name='tacaia2')
#tacaia2.loc[('B2003125AFA','LF03','33','MD2005061901'),'PRODUCT_CODE'][0][:4] #Pandas多层索引的引用
#[x[0] for x in tacaia2.index] #多层索引中一层所有元素的列表

In [ ]:
acc_period=tacaia2['ACCOUNT_PERIOD'][1]
print(acc_period)

In [ ]:
tacacm4=pd.read_sql('select * from tacacm4 where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])

In [ ]:
tacacmm=pd.read_sql('select * from tacacmm where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])

In [ ]:
tacacj5=pd.read_sql_query('select * from tacacj5 where YEAR_MON=?',conn,params=[acc_period])

In [ ]:
print(tacacj5.head())
print(tacacm4.head())
print(tacacmm.head())

In [ ]:
#将ACAI25表按1、材料号升序；2、账务代码：按33 03 31 01 50 51 的顺序排列；3、销售合同号按升序排列
#tacaia2[tacaia2['MAT_WT'].rank(method='dense',ascending=False)==3]

In [ ]:
#对账务代码自定义排序：先设排序的列表；将账务代码列的数据类型改成‘category’（astype)；用set_category，设置账务代码的顺序；用sort_values使排序生效，同时，也将材料号和销售合同升序排列
acc_ttl_list=['33','03','31','01','50','51']
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype('category')
tacaia2['ACCOUNT_TITLE_ITEM'].cat.set_category(acc_ttl_list,inplace=True)
tacaia2.sortvalues(['MAT_NO','ACCOUNT_TITLE_ITEM','SALE_ORDER_SUB_NO'],ascending=True,inplace=True)
tacaia2